Table maker

In [1]:
import json
from pathlib import Path

import pandas as pd
from matplotlib import pyplot as plt

results_dir = Path(".", "results_minimal")
results_stratified_dir = Path(".", "results_minimal_stratified")
scaling_transformations = set([str(file).split("_")[2] for file in list(results_dir.glob("*.json"))])
scaling_transformations

{'MaxAbsScaler',
 'MinMaxScaler',
 'QuantileTransformer',
 'RobustScaler',
 'StandardScaler'}

Table with results for each scaler

In [2]:
# Iterate through the scaler types
for scaler in scaling_transformations:
    # Iterate through non-stratified and stratified results
    for split_type, split_type_name in zip([results_dir, results_stratified_dir], ["nonstratified", "stratified"]):
        scaler_results = pd.DataFrame(data=None, columns=None, index=["max", "min", "mean", "std"])
        # Iterate through individual model results for the given scaler type and split type
        for result in results_dir.glob(f"*{scaler}*"):
            # Load the result
            data = pd.read_json(result).transpose()
            # Get the leakage and correct BCC values
            data = pd.json_normalize(data["bcc"])
            # Calculate the difference
            data["diff"] = data["leakage"] - data["correct"]
            # Get the metrics
            col = data[["diff"]].agg(func=["max", "min", "mean", "std"])
            # Name the column with the name of the classifier
            col.columns = [result.stem.split("_")[-1]]
            # Concatenate the column with the performance of previous results
            scaler_results = pd.concat([scaler_results, col], axis=1)
        # Transpose the results so classifiers are rows
        scaler_results = scaler_results.transpose().reset_index().rename(columns={"index": "classifier"})
        # Once all results for the split type and scaler are added, save them
        scaler_results.to_csv(f"scaler_{scaler}_split_{split_type_name}.csv", index=False)


In [ ]:
pandas_tables = []

for scaler in scaling_transformations:
    scaler_table = {"classifier": [],
                    "maxdiff": [],
                    "mindiff": [],
                    "meandiff": [],
                    "stddiff": [],
                    "bestdiff": [],
                    "bestcorrect": [],
                    "meancorrect": [],
                    "worstcorrect": []}
    for result in results_dir.glob(f"*{scaler}*"):
        with open(result, "r") as f:
            clf_results = json.load(f)
            to_pandas = {"seed": [], "bcc_correct": [], "bcc_leakage": []}
            for seed_experiment in clf_results.keys():
                to_pandas["seed"].append(int(seed_experiment))
                to_pandas["bcc_correct"].append(clf_results[seed_experiment]["bcc"]["correct"])
                to_pandas["bcc_leakage"].append(clf_results[seed_experiment]["bcc"]["leakage"])

            pd_data = pd.DataFrame(to_pandas)
            scaler_table["classifier"].append(str(result.stem).split("_")[4])
            diff = pd_data["bcc_leakage"] - pd_data["bcc_correct"]
            # plt.figure()
            # plt.hist(diff, bins=100)
            # plt.title(f"{result.stem} - {scaler}")

            scaler_table["maxdiff"].append(diff.max())
            scaler_table["mindiff"].append(diff.min())
            scaler_table["meandiff"].append(diff.mean())
            scaler_table["stddiff"].append(diff.std())
            scaler_table["bestdiff"].append(pd_data["bcc_leakage"].max() - pd_data["bcc_correct"].max())
            scaler_table["bestcorrect"].append(pd_data["bcc_correct"].max())
            scaler_table["meancorrect"].append(pd_data["bcc_correct"].mean())
            scaler_table["worstcorrect"].append(pd_data["bcc_correct"].min())

    pd_to_dump = pd.DataFrame(scaler_table).to_csv(f"scaler_{scaler}.csv")
plt.show()

In [ ]:
# Code for table setup
TABLE_START = \
"""
\\begin{table}[]
\t\\centering
\t\\begin{tabular}{lcccc}
"""

# Encoding estimator names
estimator_names = {    "adaboost":"AdaBoost",
    "dt": "\\gls{DT}",
    "gaussianNB": "Gaussian \\gls{NB}",
    "gaussian": "\\gls{GP}",
    "knn": "\\gls{KNN}",
    "lda": "\\gls{LDA}",
    "mlp": "\\gls{MLP}",
    "qda": "\\gls{QDA}",
    "rf": "\\gls{RF}",
    "svm": "\\gls{SVM}"
    }

# Encoding column names
col_names = {
    "classifier": "Estimator",
    "maxdiff": "MAX",
    "mindiff": "MIN",
    "meandiff": "$\\mu$",
    "stddiff": "$\\sigma$",
}
# Code for table end -> needs to be formatted to add the name of the tranformer
table_end = \
"""
\t\\end{{tabular}}
\t\\caption{{Data leakage results for {transformer}}}
\t\\label{{tab:{transformer_label}}}
\\end{{table}}
"""

# Getting the name of the transformer
table_latex = ""
for transformer in ["MaxAbsScaler", "MinMaxScaler", "QuantileTransformer", "RobustScaler", "StandardScaler"]:
    # Loading data
    data = pd.read_csv(f"scaler_{transformer}.csv").iloc[:, 1:6]
    # Adding a toprule
    table_header = "\t\t\\toprule\n"
    # Creating a table header from the dataframe column names
    table_header += "\t\t" + " & ".join( [col_names[x] for x in data.columns.to_list()]) + " \\\\\n"
    # Adding a midrule
    table_header += "\t\t\\midrule"
    # Creating the table body with values for each model
    table_body = ""
    for row in data.iterrows():
      # Saving only the values (discarding indices) as a list
      row_as_list = row[1].to_list()
      # Formatting the values as floats with 3 decimals
      row_formatted = [estimator_names[row_as_list[0]]] + [f"{val:.3f}" for val in row_as_list[1:]]
      # Creating a latex code for the row with values
      if row[0] == 0:
        table_row = "\n\t\t" + " & ".join(row_formatted)
      else:
        table_row = " \\\\\n\t\t" + " & ".join(row_formatted)
      # Adding the code to the existing table body
      table_body += table_row
    # Adding a bottomrule
    table_body += "\\\\\n\t\t\\bottomrule"

    #
    table_latex += TABLE_START + table_header + table_body + table_end.format(transformer=transformer, transformer_label=transformer.replace(" ","_").lower())

with open("latex_tables.tex", "w") as f:
  f.write(table_latex)